In [7]:
from arguments import ModelParams, ArgumentParser
from depth_images import calibrate_depth
from gaussian_renderer import GaussianModel
from scene import Scene
import matplotlib.pyplot as plt

parser = ArgumentParser()
dataset = ModelParams(parser)
dataset.model_path = ""
dataset.source_path = "../data/redwood_proc/00002/"
dataset.images = "images"
dataset.resolution = -1
dataset.white_background = False
dataset.eval=True
dataset.num_train_images = 6
gaussians = GaussianModel(dataset.sh_degree)
scene = Scene(dataset, gaussians,shuffle=False)

calibrate_depth(scene)

Reading camera 14/192

Reading camera 192/192
Converting point3d.bin to .ply, will happen only the first time you open the scene.
Loading Training Cameras
Loading Test Cameras
Number of points at initialisation :  3355
2.8093648193931666 -0.33844559854187223


In [8]:
import open3d as o3d
import numpy as np
from utils.graphics_utils import fov2focal

In [9]:
def camera_to_pcd(camera):
    intrinics = o3d.camera.PinholeCameraIntrinsic(
        camera.image_width,
        camera.image_height,
        500,
        500,
        camera.image_width/2,
        camera.image_height/2,
    )

    depth_image_o3d = o3d.geometry.Image(2.5/2.6*camera.depth.numpy()[0].astype(np.float32))
    pcd = o3d.geometry.PointCloud.create_from_depth_image(depth_image_o3d,intrinics,camera.world_view_transform.cpu().numpy().T)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=10, max_nn=30))
    return pcd

all_pcds = []
for i,camera in enumerate(scene.getTrainCameras()):
    pcd = camera_to_pcd(camera)
    all_pcds.append(pcd)

In [10]:
def pairwise_registration(source, target,max_correspondence_distance_coarse,
                      max_correspondence_distance_fine):
    print("Apply point-to-plane ICP")
    icp_coarse = o3d.registration.registration_icp(
        source, target, max_correspondence_distance_coarse, np.identity(4),
        o3d.registration.TransformationEstimationPointToPlane())
    icp_fine = o3d.registration.registration_icp(
        source, target, max_correspondence_distance_fine,
        icp_coarse.transformation,
        o3d.registration.TransformationEstimationPointToPlane())
    transformation_icp = icp_fine.transformation
    information_icp = o3d.registration.get_information_matrix_from_point_clouds(
        source, target, max_correspondence_distance_fine,
        icp_fine.transformation)
    return transformation_icp, information_icp


def full_registration(pcds, max_correspondence_distance_coarse,
                      max_correspondence_distance_fine):
    pose_graph = o3d.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id],max_correspondence_distance_coarse,
                      max_correspondence_distance_fine)
            print("Build o3d.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.registration.PoseGraphNode(
                        np.linalg.inv(odometry)))
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=True))
    return pose_graph

def register_pointclouds(all_pcds):
    print("Full registration ...")
    voxel_size = 1
    max_correspondence_distance_coarse = voxel_size * 15
    max_correspondence_distance_fine = voxel_size * 1.5
    #with o3d.utility.VerbosityContextManager(
    #        o3d.utility.VerbosityLevel.Debug) as cm:
    pose_graph = full_registration(all_pcds,
                                    max_correspondence_distance_coarse,
                                    max_correspondence_distance_fine)

    print("Optimizing PoseGraph ...")
    option = o3d.registration.GlobalOptimizationOption(
        max_correspondence_distance=max_correspondence_distance_fine,
        edge_prune_threshold=0.25,
        reference_node=0)
    # with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    o3d.registration.global_optimization(
        pose_graph, o3d.registration.GlobalOptimizationLevenbergMarquardt(),
        o3d.registration.GlobalOptimizationConvergenceCriteria(), option)

    for point_id in range(len(all_pcds)):
        print(pose_graph.nodes[point_id].pose)
        print(pose_graph.nodes[point_id].pose)
        all_pcds[point_id].transform(pose_graph.nodes[point_id].pose)

In [11]:
all_points = []
all_colors = []
for i,pcd in enumerate(all_pcds):
    points = np.array(pcd.points)
    colors = np.zeros_like(points)
    colors[:,i%3] = 1.0
    if i>=3:
        colors[:,(i+1)%3] = 1.0
    all_points.append(points)
    all_colors.append(colors)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.concatenate(all_points))
pcd.colors = o3d.utility.Vector3dVector(np.concatenate(all_colors))
o3d.io.write_point_cloud("test.ply", pcd)

True